# Time series predictor

In [35]:
import numpy as np
import time
from datetime import date
import csv
import holidays # for importing the public holidays
import re

bologna_holidays = holidays.Italy(prov='BO') # Get the holidays in Bologna, Italy :)

In [36]:
# Variables
data_dir = "data"
dataset = "data/LastSeenDataset.csv"

- Feature extraction: we first extract the features given the time series data of Telegram accesses.
- Supposition: last Telegram access in very similar to the time the person goes to sleep

## Features
Possible features to extract: 
1. Last seen time (arguably the most important)
2. Wake up time
3. Number of Telegram accesses during the previous day
4. Day of the week
5. Public holiday presence in the following day (using the holidays library)
6. (time spent on Telegram)


In [45]:
## OLD FUNCTIONS
'''
We don't need these functions since we have already converte
def date_time_separate(data):
    for i in range(len(data)):
        d = (re.split(' ', str(data[i][0]))[0])
        #d = d.split('-')
        t = (re.split(' ', str(data[i][0]))[1])
        #t = t.split(':')
        data[i] = [d, t]
    return data

def singe_value_split(data):
    for i in range(len(data)):
        d = (re.split(' ', str(data[i][0]))[0])
        d = d.split('-')
        t = (re.split(' ', str(data[i][0]))[1])
        t = t.split(':')
        data[i] = [d, t]
    return data
'''


"\nWe don't need these functions since we have already converte\ndef date_time_separate(data):\n    for i in range(len(data)):\n        d = (re.split(' ', str(data[i][0]))[0])\n        #d = d.split('-')\n        t = (re.split(' ', str(data[i][0]))[1])\n        #t = t.split(':')\n        data[i] = [d, t]\n    return data\n\ndef singe_value_split(data):\n    for i in range(len(data)):\n        d = (re.split(' ', str(data[i][0]))[0])\n        d = d.split('-')\n        t = (re.split(' ', str(data[i][0]))[1])\n        t = t.split(':')\n        data[i] = [d, t]\n    return data\n"

In [82]:
with open(dataset, newline='') as csvfile:
    date_list = list(csv.reader(csvfile))

# Transform all the elements in the dataset into date objects and sort them
def convert_to_dates(dates):
    for i in range(len(dates)):
        dates[i][0] = datetime.datetime.strptime(dates[i][0], "%Y-%m-%d %H:%M:%S" ) 
    return sorted(dates)

date_list = convert_to_dates(date_list)

print(date_list[0][0] in bologna_holidays)

print(date_list[0][0].year)
#print(date_list)

False
2020


In [181]:
# Detect last seen time at night

fall_asleep = []
wake_up = []
time_slept = []
day_of_week = []
holiday_presence = []
num_accesses = []

# Last seen during this time will be considered as last seen time
# BEWARE: cannot detect times out of this range
MIN_H, MIN_M, MIN_S = 21, 0, 0
MAX_H, MAX_M, MAX_S = 5, 0, 0

## Suppose detection started before MIN_HOUR
# Initialize
min_asleep_time = date_list[0][0]
min_asleep_time = min_asleep_time.replace(hour= MIN_H, minute= MIN_M, second= MIN_S)
max_asleep_time = date_list[0][0]
max_asleep_time = max_asleep_time.replace(day= date_list[0][0].day +1, hour= MAX_H, minute= MAX_M, second= MAX_S)
# max_t = min_asleep_time
# m_fall_asleep_time.hour, min_asleep_time.min, min_asleep_time.second = MIN_H, MIN_M, MIN_S

saved = True
n_accesses = 0 # initialize to 0 the counter for number of status changes

for i in range(len(date_list)):
    # If time is a candidate for falling asleep (=good night) then save it
    d = date_list[i][0]
    if d > min_asleep_time and d < max_asleep_time:
        sleeptime = d # save this time
        save = False
    else:
        if not saved:
            # We save all the data in one single time so not to create mismatches
            # Update indexes every first Telegram access in the morning
            min_asleep_time = d.replace(hour= MIN_H, minute= MIN_M, second= MIN_S)
            max_asleep_time = d.replace(day= d.day + 1, hour= MAX_H, minute= MAX_M, second= MAX_S)
            fall_asleep.append(sleeptime)
            wake_up.append(d)
            time_slept.append(d - sleeptime)
            day_of_week.append(d.weekday())
            holiday_presence(d in bologna_holidays)
            num_accesses.append(n_accesses)
            saved = True
            n_accesses = 0
    n_accesses += 1 # increase counter for every different last seen ~ number of accesses
            
print("Fall asleep time: ", fall_asleep)
print("Wake up time: ", wake_up)
print("Time slept (seconds): ", time_slept)
print("Day of the week (from 1 = Monday): ", day_of_week)
print("Holiday presence: ", holiday_presence)
print("Number of accesses ", num_accesses)

Fall asleep time:  []
Wake up time:  []
Time slept (seconds):  []
Day of the week (from 1 = Monday):  []
Holiday presence:  []
Number of accesses  []


## Model
- Time series data, so possible idea(s):
    - LSTM